Define notebook variables

In [ ]:
# @hidden_cell
# The project token is an authorization token that is used to access project resources like data sources, connections, and used by platform APIs.

project_id = ''
project_access_token = ''

training_bucket = project_id + '-training'
results_bucket = project_id + '-results'

cos_connection_name = 'COS Images'

Extract the images

In [ ]:
import keras
from keras.datasets import mnist
import pickle, numpy
from sklearn.model_selection import train_test_split

# The data, split between train and test sets:
(X, y), (X_test, y_test) = mnist.load_data()
# Make additional split between training and validation sets:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.166, random_state=42)

with open('mnist-tf-train.pkl', 'wb') as f:
    pickle.dump((X_train, y_train), f, protocol=pickle.HIGHEST_PROTOCOL)

with open('mnist-tf-valid.pkl', 'wb') as f:
    pickle.dump((X_valid, y_valid), f, protocol=pickle.HIGHEST_PROTOCOL)

with open('mnist-tf-test.pkl', 'wb') as f:
    pickle.dump((X_test, y_test), f, protocol=pickle.HIGHEST_PROTOCOL)

Verify that they have been created locally

In [ ]:
!ls -lh

Get the COS Connection credentials

In [ ]:
from project_lib import Project
project = Project(project_id=project_id, project_access_token=project_access_token)
pc = project.project_context

cos_asset_id = [ d['asset_id'] for d in project.get_assets()if d['name'] == cos_connection_name and d['type'] == 'connection' ][0]
cos_creds = project.get_connection(id=cos_asset_id )

In [ ]:
! pip install --quiet ibm-cos-sdk

Create COS buckets and upload images to training bucket

In [ ]:
import ibm_boto3
from ibm_botocore.client import Config

api_key = cos_creds['api_key']
service_instance_id = cos_creds['resource_instance_id']
auth_endpoint = cos_creds['iam_url']
service_endpoint = 'https://' + cos_creds['url']

cos = ibm_boto3.client('s3',
                      ibm_api_key_id=api_key,
                      ibm_service_instance_id=service_instance_id,
                      ibm_auth_endpoint=auth_endpoint,
                      config=Config(signature_version='oauth'),
                      endpoint_url=service_endpoint)

# Create bucket if it doesn't exist
for bucket in [ training_bucket, results_bucket ]:
    try:
        cos.create_bucket(Bucket=bucket)
        print('Created bucket: ' + bucket)
    except:
        print('Bucket already exist: ' + bucket)
        
print('--')
        
for f in ['mnist-tf-train.pkl', 'mnist-tf-valid.pkl', 'mnist-tf-test.pkl']:
    with open(f, 'rb') as data:
       cos.upload_fileobj(data, training_bucket, f)
       print('Uploaded {} to {}'.format(f, training_bucket))